<a href="https://colab.research.google.com/github/MarufAnsari/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_the_city_of_Toronto%2C_Canada_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segmenting and Clustering Neighborhoods in the city of Toronto, Canada**

##**PART 1**

**1. Importing Libraries**

In [23]:
import numpy as np                # library to handle data in a vectorized manner
import pandas as pd               # library for data analsysis
import requests                   # library to handle requests
from bs4 import BeautifulSoup     # library to parse HTML and XML documents

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json                                   # library to handle JSON files
from geopy.geocoders import Nominatim         # convert an address into latitude and longitude values
from pandas.io.json import json_normalize     # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print('Libraries imported.')

Libraries imported.


**2. Scrap data from Wikipedia page into a DataFrame**

Get the HTML page of Wikipedia showing the data of Toronto city.

We will use read_html function here in order to convert the html data into list of Dataframe objects.

We will remove the rows with not assigned boroughs

In [30]:
# requesting to get the html page from the URL/Link
wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(wiki_page, 'html.parser') 

#Lists to store the table data
PostalCode_ = []
Borough_ = []
Neighborhood_ = []

#appending data into lists
for row in soup.find('table').find_all('tr') :
  cells = row.find_all('td')
  if(len(cells)>0) :
    PostalCode_.append(cells[0].text.rstrip('\n'))
    Borough_.append(cells[1].text.rstrip('\n'))
    Neighborhood_.append(cells[2].text.rstrip('\n'))

#creating Dataframe by concatconcatenating the lists
toronto_df = pd.DataFrame({"PostalCode":PostalCode_, "Borough":Borough_, "Neighborhood":Neighborhood_})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**3. Dropping the cells with a borough that is "Not assigned"**



In [33]:
# dropping "Not assigned" borough
toronto_df['Borough'].replace('Not assigned', np.nan, inplace=True)
toronto_df.dropna(subset=['Borough'], inplace=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**4. Group Neighborhoods in the same borough**

In [43]:
# Grouping by Borough and Postaal-Code
toronto_df_grp = toronto_df.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
toronto_df_grp.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**5. "Not assigned" Neighborhood is replaced with respective borough name**

In [44]:
# replacing "Not assigned" Neighborhood
for index, row in toronto_df_grp.iterrows() :
  if row['Neighborhood'] == 'Not assigned' :
    row['Neighborhood'] = row['Borough']

toronto_df_grp.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**6. Matching the table with the given table in question**

In [52]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

postcode_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in postcode_list:
    test_df = test_df.append(toronto_df_grp[toronto_df_grp["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [63]:
# Shape of our table
toronto_df_grp.shape

(103, 3)

**PART 2**

**1. Load co-ordinates from the csv file of coursera**

In [58]:
#loading co-ordinates
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.columns = ['PostalCode', 'Latitude', 'Longitude']
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**2. Merge two tables of data and co-ordinates on Postal-Code**

In [60]:
toronto_df_geo = toronto_df_grp.merge(geo_df, on='PostalCode', how='left')
toronto_df_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**3. Matching the table with the given table in question**

In [62]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

postcode_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in postcode_list:
    test_df = test_df.append(toronto_df_geo[toronto_df_geo["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [64]:
# Shape of the new table with respective co-ordinates
toronto_df_geo.shape

(103, 5)

##**PART 3**

**1. Use geopy library to get the co-ordinates of Toronto**

In [66]:
#getting geographical co-ordinates
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical co-ordinates of Toronto is {}, {}".format(latitude, longitude))

The geographical co-ordinates of Toronto is 43.6534817, -79.3839347


**2. Create a map of Toronto with Neighborhoods superimposed on the top**

In [67]:
# plotting map of Toronto based on co-ordinates
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_geo['Latitude'],toronto_df_geo['Longitude'],toronto_df_geo['Borough'],toronto_df_geo['Neighborhood']):
  label = '{},{}'.format(neighborhood,borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7).add_to(map_toronto)

map_toronto

**3. Filter the boroughs that have the word Toronto**

In [70]:
toronto_df_new = toronto_df_geo[toronto_df_geo['Borough'].str.contains('Toronto')]
toronto_df_new = toronto_df_new.reset_index(drop=True)
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


**4. Map of Toronto with borough having name containing toronto**

In [71]:
# plotting new map of Toronto based on co-ordinates
map_toronto_new = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'],toronto_df_new['Longitude'],toronto_df_new['Borough'],toronto_df_new['Neighborhood']):
  label = '{},{}'.format(neighborhood,borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7).add_to(map_toronto_new)

map_toronto_new

**5. Use Foursquare API to explore the neighborhoods**

In [146]:
# Define foursquare credentials and version
CLIENT_ID = 'client_id'
CLIENT_SECRET = 'client_secret'
VERSION = '20180605'    # API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: client_id
CLIENT_SECRET:client_secret


Getting the location and name of first neighborhood

In [74]:
first_lat = toronto_df_new.loc[0,'Latitude']
first_lng = toronto_df_new.loc[0, 'Longitude']
first_neigh = toronto_df_new.loc[0, 'Neighborhood']
print("The first neighborhood of new table of Toronto city is {} with co-ordinates {}, {}".format(first_neigh,first_lat,first_lng))

The first neighborhood of new table of Toronto city is The Beaches with co-ordinates 43.67635739999999, -79.2930312


Let's explore top 100 venues near this location within 500m radius

In [75]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_lat,
    first_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=F0CGGFPMVAL3GVT0QRCNQBICT5ECO2AS1JBAGXLGHPFAEYI3&client_secret=TEHXGMHAFUH5BIKCW4LYLS2V2NYENO3L2SXXPURDGSANLRJM&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [77]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5f1c2f5b5dc8d450a1860cfb'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

Now Let's get the top 100 venues for all the neighborhoods of the Toronto city within a radius of 500m

In [80]:
LIMIT = 100
radius = 500
venues = []

for lat, lng, post, borough, neighborhood in zip(toronto_df_new['Latitude'],toronto_df_new['Longitude'],
                                                toronto_df_new['PostalCode'],toronto_df_new['Borough'],
                                                toronto_df_new['Neighborhood']) :
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        for ven in results :
          venues.append((post, borough, neighborhood, lat, lng,
                         ven['venue']['name'], ven['venue']['location']['lat'],
                         ven['venue']['location']['lng'], ven['venue']['categories'][0]['name']))

In [81]:
# convert the venue list into a dataframe
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'Borough_Latitude', 'Borough_Longitude', 'Venue_name','Venue_Latitude', 'Venue_Longitude', 'Venue_Category']
print(venues_df.shape)
print("\n")
venues_df.head(10)

(1637, 9)




,PostalCode,Borough,Neighborhood,Borough_Latitude,Borough_Longitude,Venue_name,Venue_Latitude,Venue_Longitude,Venue_Category
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
5,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
6,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
7,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677702,-79.352265,Ice Cream Shop
8,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
9,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


Let's check number of venues near each neighborhood

In [83]:
  # use count method to get the number of venues for each neighborhood
  venues_df.groupby(['PostalCode','Borough', 'Neighborhood']).count()

,,,Borough_Latitude,Borough_Longitude,Venue_name,Venue_Latitude,Venue_Longitude,Venue_Category
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,5,5,5,5,5,5
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",18,18,18,18,18,18
M4M,East Toronto,Studio District,41,41,41,41,41,41
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,10,10,10,10,10,10
M4R,Central Toronto,"North Toronto West, Lawrence Park",17,17,17,17,17,17
M4S,Central Toronto,Davisville,35,35,35,35,35,35
M4T,Central Toronto,"Moore Park, Summerhill East",2,2,2,2,2,2


Let's find out the number of unique venue categories

In [84]:
print("There are", len(venues_df['Venue_Category'].unique()), "unique venue categories")

There are 231 unique venue categories


In [89]:
venues_df['Venue_Category'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Asian Restaurant', 'Cosmetics Shop', 'Greek Restaurant',
       'Ice Cream Shop', 'Italian Restaurant', 'Brewery', 'Juice Bar',
       'Yoga Studio', 'Restaurant', 'Fruit & Vegetable Store',
       'Pizza Place', 'Bookstore', 'Dessert Shop',
       'Furniture / Home Store', 'Bubble Tea Shop', 'Spa',
       'Grocery Store', 'Café', 'Bakery', 'Coffee Shop',
       'Caribbean Restaurant', 'Indian Restaurant', 'Lounge',
       'American Restaurant', 'Frozen Yogurt Shop', 'Sushi Restaurant',
       'Liquor Store', 'Gym', 'Fast Food Restaurant', 'Fish & Chips Shop',
       'Park', 'Burrito Place', 'Pet Store', 'Steakhouse',
       'Movie Theater', 'Sandwich Place', 'Board Shop', 'Fish Market',
       'Gay Bar', 'Cheese Shop', 'Seafood Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store',
       'Comfort Food Restaurant', 'Thai Restaurant', 'Coworking Space'],
      dtype=object)

**6. Analyze Each Neighborhood**

In [94]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['Venue_Category']], prefix="", prefix_sep="")

# add postal-code, borough, neighborhood back to the dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode']
toronto_onehot['Borough'] = venues_df['Borough']
toronto_onehot['Neighborhood'] = venues_df['Neighborhood']

# move postal-code, borough, neighborhood to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1637, 233)


,Yoga Studio,PostalCode,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,M4E,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,M4E,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [98]:
#grouping neighborhoods and taking mean of theirb frequency
toronto_onehot_grp = toronto_onehot.groupby(['PostalCode','Borough','Neighborhood']).mean().reset_index()
print(toronto_onehot_grp.shape)
toronto_onehot_grp

(39, 233)


,PostalCode,Borough,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.200000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

Now Let's display the top 10 venues for each Postal-Code

In [132]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

#create columns according to the number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhood']
freqColumns = []
for ind in np.arange(num_top_venues) :
  try : 
    freqColumns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
  except : 
    freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new Data Frame
neighborhood_venues_sorted = pd.DataFrame(columns = columns)
neighborhood_venues_sorted['PostalCode'] = toronto_onehot_grp['PostalCode']
neighborhood_venues_sorted['Borough'] = toronto_onehot_grp['Borough']
neighborhood_venues_sorted['Neighborhood'] = toronto_onehot_grp['Neighborhood']

for ind in np.arange(toronto_onehot_grp.shape[0]) :
  row_categories = toronto_onehot_grp.iloc[ind,:].iloc[3:]
  row_categories_sorted = row_categories.sort_values(ascending=False)
  neighborhood_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

#neighborhood_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhood_venues_sorted.shape)
neighborhood_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Asian Restaurant,Health Food Store,Pub,Trail,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Women's Store
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Pub,Indian Restaurant,Spa,Dessert Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",Pet Store,Fish & Chips Shop,Sandwich Place,Board Shop,Brewery,Restaurant,Italian Restaurant,Burrito Place,Ice Cream Shop,Steakhouse
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop,Clothing Store
4,M4N,Central Toronto,Lawrence Park,Photography Studio,Bus Line,Park,Swim School,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
5,M4P,Central Toronto,Davisville North,Hotel,Park,Food & Drink Shop,Pizza Place,Breakfast Spot,Gym,Gym / Fitness Center,Department Store,Sandwich Place,Dessert Shop
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Yoga Studio,Fast Food Restaurant,Chinese Restaurant,Cosmetics Shop,Café,Restaurant,Mexican Restaurant,Salon / Barbershop
7,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Coffee Shop,Park,Brewery
8,M4T,Central Toronto,"Moore Park, Summerhill East",Gym,Tennis Court,Women's Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Sushi Restaurant,Fried Chicken Joint,Liquor Store,Restaurant,Bank,Sports Bar,Bagel Shop,Supermarket


**7. Cluster Neighborhood**

Run *k*-means to cluster the neighborhood into 5 clusters

In [133]:
# set of clusters
kclusters = 5
toronto_grp_clustering = toronto_onehot_grp.drop(['PostalCode','Borough','Neighborhood'],1)

# run kmeans clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grp_clustering)

# check cluster label generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 0, 0, 3, 0], dtype=int32)

In [136]:
# create a new dataframe that includes the cluster labels along wit ]h top 10 venues for each neighborhood
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhood_venues_sorted.drop(["Borough", "Neighborhood"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head()

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Health Food Store,Pub,Trail,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Women's Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Pub,Indian Restaurant,Spa,Dessert Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Pet Store,Fish & Chips Shop,Sandwich Place,Board Shop,Brewery,Restaurant,Italian Restaurant,Burrito Place,Ice Cream Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Bus Line,Park,Swim School,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [137]:
# sort the results by cluster label
toronto_merged.sort_values(['Cluster Labels'], inplace=True)
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Health Food Store,Pub,Trail,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Women's Store
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,American Restaurant,Salad Place,Japanese Restaurant,Beer Bar,Sporting Goods Shop
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Sandwich Place,Café,Coffee Shop,Donut Shop,History Museum,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,BBQ Joint
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Restaurant,Bookstore,Bar,Japanese Restaurant,Bakery,Sandwich Place,Beer Bar,Beer Store,Italian Restaurant
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Bar,Mexican Restaurant,Dessert Shop,Park,Dumpling Restaurant,Pizza Place
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Bar,Rental Car Location,Sculpture Garden,Plane,Coffee Shop
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Beer Bar,Seafood Restaurant,Japanese Restaurant,Hotel,Gym,Bakery
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Seafood Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Diner,Candy Store,Italian Restaurant,Nightclub,Restaurant,Baby Store,Coffee Shop


Finally let's visualise the resulting clusters

In [139]:
# create a map
map_clusters = folium.Map(location=[latitude,longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon , post, bor, neigh, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],
                                                toronto_merged['PostalCode'],toronto_merged['Borough'],
                                                toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']) :
                                                label = folium.Popup("{} ({}) : {} - Cluster {}".format(bor,post,neigh,cluster), parse_html=True)
                                                folium.CircleMarker([lat,lon], radius=5,
                                                                    popup=label, color=rainbow[cluster-1],
                                                                    fill=True, fill_color=rainbow[cluster-1],
                                                                    fill_opacity=0.7).add_to(map_clusters)
map_clusters

**8. Examine Clusters**

Cluster no. 1

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels']==0, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Asian Restaurant,Health Food Store,Pub,Trail,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Women's Store
20,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,American Restaurant,Salad Place,Japanese Restaurant,Beer Bar,Sporting Goods Shop
21,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Donut Shop,History Museum,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,BBQ Joint
25,Downtown Toronto,0,Café,Restaurant,Bookstore,Bar,Japanese Restaurant,Bakery,Sandwich Place,Beer Bar,Beer Store,Italian Restaurant
26,Downtown Toronto,0,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Bar,Mexican Restaurant,Dessert Shop,Park,Dumpling Restaurant,Pizza Place
27,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Bar,Rental Car Location,Sculpture Garden,Plane,Coffee Shop
28,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Beer Bar,Seafood Restaurant,Japanese Restaurant,Hotel,Gym,Bakery
29,Downtown Toronto,0,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Seafood Restaurant
30,Downtown Toronto,0,Grocery Store,Café,Park,Diner,Candy Store,Italian Restaurant,Nightclub,Restaurant,Baby Store,Coffee Shop


Cluster no. 2

In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels']==1, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Pool,Garden,Women's Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


Cluster no. 3

In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels']==2, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Photography Studio,Bus Line,Park,Swim School,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
10,Downtown Toronto,2,Park,Playground,Trail,Women's Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


Cluster no. 4

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels']==3, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Gym,Tennis Court,Women's Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


Cluster no. 5

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels']==4, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Women's Store,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


**Conclusion**

Most of the neighborhoods lies in Cluster 1 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2 is just a garden, Cluster 3 are playground and park, Cluster 4 are park and swim school, and lastly Cluster 5 are park and trail.

**Thank You**